In [14]:
from sklearn.model_selection import train_test_split
import time
import networkx as nx




In [27]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd

import LP
from sklearn.model_selection import KFold
import time



class LocalSimilarity():
    def __init__(self, names, path_to_save, func, model):
        for n in names:
            G = nx.read_gml(f'CUDA_dataset/GMLs/{n}.gml')
            
            _ = self.LP_exmperiment1(G,n, func)            

    def LP_exmperiment1(self, G, n, func):
        Nodes = list(G.nodes())
        print(f'')

        E = list(G.edges())
        
        # Split the edges into train and validation sets
        train_data, val_data = train_test_split(E, test_size=0.2)  # Adjust test_size as needed

        start_time = time.time()

        gT, gP = nx.Graph(), nx.Graph()
        gT.add_nodes_from(Nodes)
        gP.add_nodes_from(Nodes)

        gT.add_edges_from(train_data)
        gP.add_edges_from(val_data)

        S = []
        for u in gT.nodes():
            for v in gT.nodes():
                if not gT.has_edge(u, v):
                    Nv, Nu = list(gT.neighbors(v)), list(gT.neighbors(u))
                    Sxy = func(gT, v, u, Nv, Nu)
                    y = 1 if gP.has_edge(v, u) else 0

                    S.append([Sxy, y])

        end_time = time.time() - start_time
        res, y = zip(*S)
        print(f'{n},Nodes,{len(Nodes)},Edges,{len(G.edges)},time,{end_time},AUC,{LP.get_AUC(y, res)}')
    
    
    def LP_exmperiment(self, G, n, func):
        Nodes = list(G.nodes())
        # U = nx.Graph()
        # U.add_nodes_from(Nodes)
        # U.add_edges_from(LP.getU(list(G.nodes())))
        # eU = list(U.edges())
        print(f'Nodes = {len(Nodes)}\nEdges = {len(G.edges)}')
        # ------------------------------------------------
        S = []
        
        kfold = KFold(n_splits=5, shuffle=True)
        E = list(G.edges())
        i = 1
        for train_index, val_index in kfold.split(E)[0]:
            start_time = time.time()
            train_data = [E[i] for i in train_index]
            val_data   = [E[i] for i in val_index]

            # print(len(train_data), len(val_data))

            # exit(1)
            gT, gP = nx.Graph(), nx.Graph()
                
            gT.add_nodes_from(Nodes)
            gP.add_nodes_from(Nodes)
            
            gT.add_edges_from(train_data)
            gP.add_edges_from(val_data)

            # ------------------------------------------------
            # for v,u in L:
            for u in gT.nodes():
                for v in gT.nodes():
                    if not gT.has_edge(u,v):
                        Nv, Nu  = list(gT.neighbors(v)), list(gT.neighbors(u))
                        Sxy = func(gT, v, u, Nv, Nu)
                        y = 1 if gP.has_edge(v,u) else 0

                        S.append([Sxy, y])

            end_time = time.time() - start_time
            res, y = zip(*S)
            print(f'time consumed is: {end_time} seconds...AUC = {LP.get_AUC(y, res)}')

            # print(f'{n} \t iteration number :{i}\t{len(train_data)}\t{len(val_data)}\t{len(L)}')
            i+=1



In [16]:
def read_graph(filePath):
    G = nx.Graph()
    with open(filePath, 'r') as file:
        next(file)  # Skip the header line (if there is one)
        for line in file:
            node1, node2 = line.strip().split(',')
            G.add_edge(int(node1), int(node2))
    return G

def convert_graph(G, edges_file_path):
    with open(edges_file_path, 'w') as file:
        for edge in G.edges():
            file.write(f"{edge[0]} {edge[1]}\n")
            
def convert_edges_to_GML(filename):
    # Create an empty graph
    G = nx.Graph()

    # Read the .edges file and add edges to the graph
    with open(f'{filename}.edges', 'r') as file:
        for line in file:
            node1, node2 = line.strip().split()  # Assuming space-separated values
            G.add_edge(int(node1), int(node2))

    # Now save the graph to a GML file
    gml_file_path = f'{filename}.gml'  # Replace with your desired output path
    nx.write_gml(G, gml_file_path)



names = [
    'artist_edges',
    'athletes_edges',
    'company_edges',
    'government_edges',
    'new_sites_edges',
    'politician_edges',
    'public_figure_edges',
    'tvshow_edges'
]

facebook2 = [
    '1912',
    '107',
    '1684',
    '3437',
    '348',
    '0',
    '414',
    '686',
    '698',
    '3980'    
]

for f in facebook2:
    convert_edges_to_GML(f'CUDA_dataset/edges/facebook1/{f}')
    
# for n in names:
#     nx.write_gml(read_graph(f'{filePath}/{n}.csv'), f'{filePath}/GMLs/{n}.gml')


# for n in names:
#     print(f'workin on {n}')
#     G = nx.read_gml(f'CUDA_dataset/GMLs/{n}.gml')
#     convert_graph(G, f'CUDA_dataset/edges/{n}.edges')    



In [22]:
filePath = "CUDA_dataset/results"
_ = LocalSimilarity(names[5:], filePath, LP.CN, 'CN')


Working on politician_edges
Nodes = 5908
Edges = 41729
time consumed is: 104.99154925346375 seconds...AUC = 0.5012131242938754
Network politician_edges completed...

Working on public_figure_edges
Nodes = 11565
Edges = 67114
time consumed is: 412.5483057498932 seconds...AUC = 0.49705744664813256
Network public_figure_edges completed...

Working on tvshow_edges
Nodes = 3892
Edges = 17262
time consumed is: 41.84848618507385 seconds...AUC = 0.4964393636690764
Network tvshow_edges completed...



In [23]:
_ = LocalSimilarity(names[1:5], filePath, LP.CN, 'CN')

Working on athletes_edges
Nodes = 13866
Edges = 86858
time consumed is: 593.1397354602814 seconds...AUC = 0.505723467179865
Network athletes_edges completed...

Working on company_edges
Nodes = 14113
Edges = 52310
time consumed is: 583.2835941314697 seconds...AUC = 0.4999852810103137
Network company_edges completed...

Working on government_edges
Nodes = 7057
Edges = 89455
time consumed is: 143.78611135482788 seconds...AUC = 0.499950907817296
Network government_edges completed...

Working on new_sites_edges
Nodes = 27917
Edges = 206259
time consumed is: 2574.571785211563 seconds...AUC = 0.5006127655130799
Network new_sites_edges completed...



In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.RA, 'RA')

In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.PA, 'PA')

In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.JA, 'JA')

In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.SA, 'SA')

In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.HPI, 'HPI')

In [28]:
_ = LocalSimilarity(facebook2, filePath, LP.AA, 'AA')


1912,Nodes,747,Edges,30025,time,1.7127735614776611,AUC,0.49740720104060954

107,Nodes,1034,Edges,26749,time,3.1168534755706787,AUC,0.507737187289643

1684,Nodes,786,Edges,14024,time,1.6979007720947266,AUC,0.5183678044449984

3437,Nodes,534,Edges,4813,time,0.7118735313415527,AUC,0.5053216572429332

348,Nodes,224,Edges,3192,time,0.11908411979675293,AUC,0.502399359927682

0,Nodes,333,Edges,2519,time,0.2634899616241455,AUC,0.5189306232045554

414,Nodes,150,Edges,1693,time,0.05012869834899902,AUC,0.5071040917976074

686,Nodes,168,Edges,1656,time,0.06427955627441406,AUC,0.480420477671766

698,Nodes,61,Edges,270,time,0.007980585098266602,AUC,0.4711161176895222

3980,Nodes,52,Edges,146,time,0.005849361419677734,AUC,0.4357863460475401


In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.HDI, 'HDI')

In [ ]:
_ = LocalSimilarity(facebook2, filePath, LP.LLHN, 'LLHN')